In [136]:
import pandas as pd
from enum import Enum
import warnings

start_row = 1
end_row = 223

toolbox_original = pd.read_csv("toolbox.csv")
master = pd.read_csv("master.csv")

class Colors(Enum):
    RED = 1 # none of the below
    YELLOW = 2 # no leads, but said "no" on a product they already use
    GREEN = 3 # has leads
    BLUE = 4 # has leads, but also said "no" on a product they already use

# toolbox -> master
product_map = {
    "TMPP": "RepairLink",
    "TCPP": "CollisionLink",
    "PSX": "PSXLink (Professional)",
    "OPSTM": "TraxMechanical",
    "OPSTC": "Trax Collision"
}

toolbox = toolbox_original.copy()
toolbox["lead"] = ""

# Convert 'Dealer Code' fields to string and strip any spaces
toolbox_original['Dealer Code'] = toolbox_original['Dealer Code'].astype(str).str.strip()
master['Dealer Code'] = master['Dealer Code'].astype(str).str.strip()

def remove_leading_zero(x):
    if len(x) > 1:
        while x[0] == '0':
            if len(x) == 1:
                break
            x = x[1:]
    return x

# master has leading zeroes but toolbox doesn't
master['Dealer Code'] = master['Dealer Code'].apply(remove_leading_zero)

def evaluate_lead(toolbox_row):
    dealer_code = str(toolbox_row['Dealer Code'])
    master_row = master[master['Dealer Code'] == dealer_code]

    if master_row.empty:
        warnings.warn(f"Dealer code {dealer_code} not found in master DataFrame")
        return None, None, None 

    lead = ""
    color = Colors.RED.value
    products_using = []
    products_not_using = []
    is_canceling = False
    has_lead = False
    inquiries = {}

    for toolbox_col, master_col in product_map.items():
        inquiry = toolbox_row[toolbox_col]
        product_inquiry = inquiry == "Yes"
        product_in_use = str(master_row[master_col].values[0]) not in [None, '', 'n/a', "nan"]
        
        inquiries[toolbox_col] = inquiry

        if not product_inquiry and product_in_use:
            is_canceling = True

        if product_inquiry and not product_in_use:
            has_lead = True
            lead = master_col

        if not product_in_use:
            products_not_using.append(toolbox_col)
        if product_in_use:
            products_using.append(toolbox_col)

    if has_lead and is_canceling:
        color = Colors.BLUE.value
    elif has_lead:
        color = Colors.GREEN.value
    elif is_canceling:
        color = Colors.YELLOW.value

    using_desc, using_products = "Has" if products_using else "", ", ".join(products_using)
    not_using_desc, not_using_products = "Doesn't have" if products_not_using else "", ", ".join(products_not_using)
    desc = f"{using_desc}: {using_products}. {not_using_desc}: {not_using_products}"

    product_desc = ' '.join(f'{k}: {v}' for k, v in inquiries.items())
    desc += f" Lead: wholesale toolbox: description: {product_desc}"

    while desc[0] in [":", ",", ".", " "]:
        desc = desc[1:]

    return lead, color, desc

leads, colors, desc = zip(*[tpl for tpl in toolbox[start_row:end_row].apply(evaluate_lead, axis=1) if tpl[0] is not None])

results = toolbox[start_row:end_row].apply(evaluate_lead, axis=1)

for idx, (lead, color, desc) in zip(range(start_row, end_row), results):
    toolbox.loc[idx, 'lead'] = lead
    toolbox.loc[idx, 'color'] = color
    toolbox.loc[idx, 'Description'] = desc

toolbox.to_csv("toolbox_copy.csv", index=False)

color_map = {color.value: str(color).split(".")[-1] for color in Colors}

for color, data in toolbox.groupby('color'):
    if color:
        color_name = color_map[color]
        data.to_csv(f"toolbox_copy_{color_name}.csv", index=False)

/var/folders/fc/tb6qfjb50qld46m88qsj7r6h0000gn/T/ipykernel_3347/3219113048.py:49: UserWarning: Dealer code 4915 not found in master DataFrame
  warnings.warn(f"Dealer code {dealer_code} not found in master DataFrame")
/var/folders/fc/tb6qfjb50qld46m88qsj7r6h0000gn/T/ipykernel_3347/3219113048.py:49: UserWarning: Dealer code 29073 not found in master DataFrame
  warnings.warn(f"Dealer code {dealer_code} not found in master DataFrame")
/var/folders/fc/tb6qfjb50qld46m88qsj7r6h0000gn/T/ipykernel_3347/3219113048.py:49: UserWarning: Dealer code 41065 not found in master DataFrame
  warnings.warn(f"Dealer code {dealer_code} not found in master DataFrame")
/var/folders/fc/tb6qfjb50qld46m88qsj7r6h0000gn/T/ipykernel_3347/3219113048.py:49: UserWarning: Dealer code 4309 not found in master DataFrame
  warnings.warn(f"Dealer code {dealer_code} not found in master DataFrame")
/var/folders/fc/tb6qfjb50qld46m88qsj7r6h0000gn/T/ipykernel_3347/3219113048.py:49: UserWarning: Dealer code 25067 not found in 